In [1]:
import sys
import yaml
with open('..//secrets.yml', 'r') as file:
    secrets = yaml.safe_load(file)
sys.path.append(secrets['elo_proj_path'])

In [3]:
from player_club_classes import team_elo, Player, Club, Match
import pandas as pd
import numpy as np
import plotly.express as px
import plotly.graph_objects as go
import plotly
import pickle

from team_colors import team_color_dict
team_colors = pd.DataFrame(team_color_dict).T
team_colors.columns = ['Primary', 'Secondary']
team_colors = team_colors.rename_axis('Team').reset_index()

from plotly.validators.scatter.marker import SymbolValidator
raw_symbols = SymbolValidator().values

simple_symbols = [i for i in raw_symbols if str(i).isalpha()]

with open('../../Rugby_ELO/processed_data/playerbase.pickle', 'rb') as handle:
    playerbase = pickle.load(handle)
with open('../../Rugby_ELO/processed_data/matchlist.pickle', 'rb') as handle:
    matchlist = pickle.load(handle)
with open('../../Rugby_ELO/processed_data/teamlist.pickle', 'rb') as handle:
    teamlist = pickle.load(handle)

In [4]:
match_list = []
for _, match in matchlist.items():
    match_list.append({key:val for key, val in vars(match).items() if key not in ['home_team', 'away_team']})

match_df = pd.DataFrame(match_list)
match_df['week_num'] = match_df['date'].dt.isocalendar().week
match_df['month'] = match_df['date'].dt.month
match_df['year'] = match_df['date'].dt.year

In [16]:
## Projections
future_games = match_df[match_df.point_diff.isna()]
future_game = future_games.iloc[1]
future_game

home_team_name                                 South Africa
away_team_name                                        Wales
name                     South Africa V Wales on 2022/07/08
date                                    2022-07-08 17:05:00
competition                   International Test Match 2022
GLOBAL_home_advantage                                     0
GLOBAL_score_factor                                      10
GLOBAL_k                                                 20
GLOBAL_mov_score_add                                      1
GLOBAL_mov_score_mult                                   2.2
GLOBAL_mov_elo_add                                      2.2
GLOBAL_mov_elo_mult                                   0.001
lineup_home_elo                                 1583.617972
lineup_away_elo                                 1618.951372
lineup_elo_diff                                    -35.3334
lineup_spread                                      -3.53334
lineup_prediction                       

In [39]:
future_game["date"].date().strftime("%Y-%m-%d")

'2022-07-08'

In [30]:
f'{future_game["away_team_name"]} ({round(future_game["lineup_away_elo"], 2)}) at {future_game["home_team_name"]} ({round(future_game["lineup_home_elo"], 2)})'
favorite = future_game["home_team_name"] if future_game["lineup_spread"] > 0 else future_game["away_team_name"]
f'Prediction: {favorite} by {round(abs(future_game["lineup_spread"]), 1)}'

'Prediction: Wales by 3.5'

0.47967167180130527

In [6]:
month = match_df[(match_df.year == 2022) & (match_df.month == 7)]
closest_games = month.sort_values('lineup_spread',  key=abs).head()

sorted_pred_accuracy = month.assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs)#.drop(columns="tmp")
best_preds = sorted_pred_accuracy.head()
worst_preds = sorted_pred_accuracy.tail()

biggest_upsets = month[(month.lineup_spread * month.point_diff) < 0].assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs, ascending=False).drop(columns="tmp").head()
biggest_covers = month[((month.lineup_spread * month.point_diff) > 0)].assign(tmp=month["point_diff"] - month["lineup_spread"]).sort_values(by="tmp", key=abs, ascending=False)#.drop(columns="tmp").head()

In [8]:
player_elo_list = []
for player_name, player in playerbase.items():
    player_elo = pd.DataFrame(player.elo_list, columns = [
        'Number', 'Full_Name', 'Team', 'Player', 'Position', 'Tries',
        'Try Assists', 'Conversion Goals', 'Penalty Goals',
        'Drop Goals Converted', 'Points', 'Passes', 'Runs', 'Meters Run',
        'Clean Breaks', 'Defenders Beaten', 'Offload', 'Turnovers Conceded',
        'Tackles', 'Missed Tackles', 'Lineouts Won', 'Penalties Conceded',
        'Yellow Cards', 'Red Cards', 'espn_id_num', 'Competition', 'Date',
        'Home Team', 'Home Score', 'Away Team', 'Away Score', 'Minutes',
        'Position_Number', 'gameid', 'Unicode_ID', 'start_elo', 'end_elo'
       ])
    player_elo['Full Name'] = player_name[0]
    player_elo['Unicode_ID'] = player_name[1]
    player_elo_list.append(player_elo)

player_elo = pd.concat(player_elo_list).reset_index(drop=True)
player_elo = pd.merge(player_elo, team_colors, on = 'Team', how = 'left')
player_elo['elo_change'] = player_elo.end_elo - player_elo.start_elo
player_elo.Date = pd.to_datetime(player_elo.Date)

player_elo['week_num'] = player_elo['Date'].dt.isocalendar().week
player_elo['month'] = player_elo['Date'].dt.month
player_elo['year'] = player_elo['Date'].dt.year

In [9]:
player_elo.tail()

,Number,Full_Name,Team,Player,Position,Tries,Try Assists,Conversion Goals,Penalty Goals,Drop Goals Converted,...,Unicode_ID,start_elo,end_elo,Full Name,Primary,Secondary,elo_change,week_num,month,year
503976,5.0,Konstantine Mikautadze,Georgia,K Mikautadze,L,0.0,0.0,0.0,0.0,0.0,...,1101119510111511211095105100327511111011511697...,80,81.113136,Konstantine Mikautadze,#551828,#231f20,1.113136,26,7,2022
503977,14.0,Aka Tabutsadze,Georgia,A Tabutsadze,W,0.0,0.0,0.0,0.0,0.0,...,1101119510111511211095105100326510797328497981...,80,81.113136,Aka Tabutsadze,#551828,#231f20,1.113136,26,7,2022
503978,20.0,Luka Ivanishvili,Georgia,L Ivanishvili,NaN,0.0,0.0,0.0,0.0,0.0,...,1101119510111511211095105100327611710797327311...,80,80.0,Luka Ivanishvili,#551828,#231f20,0.0,26,7,2022
503979,11.0,Alexander Todua,Georgia,A Todua,W,0.0,0.0,0.0,0.0,0.0,...,1101119510111511211095105100326510810112097110...,80,81.113136,Alexander Todua,#551828,#231f20,1.113136,26,7,2022
503980,17.0,Nika Khatiashvili,Georgia,N Khatiashvili,NaN,0.0,0.0,0.0,0.0,0.0,...,1101119510111511211095105100327810510797327510...,80,80.0,Nika Khatiashvili,#551828,#231f20,0.0,26,7,2022


In [7]:
# CHANGE
player_month = player_elo[(player_elo.year == 2022) & (player_elo.month == 5)]
oneplayer = player_month[(player_month.Unicode_ID == '5155544848') & (player_month['Full Name'] == 'Caleb Delany')].sort_values('Date')
oneplayer.iloc[-1].end_elo - oneplayer.iloc[0].start_elo

-2.554117441069806

In [11]:
def elo_change(df):
    return df.iloc[-1].end_elo, df.iloc[0].start_elo, df.iloc[-1].end_elo - df.iloc[0].start_elo

elo_changes = player_month.sort_values('Date').groupby(['Unicode_ID', 'Full Name', 'Team']).apply(elo_change).reset_index()
elo_changes[['end_elo', 'start_elo', 'elo_change']] = pd.DataFrame(elo_changes[0].tolist(), index=elo_changes.index)
elo_changes.sort_values('elo_change')

,Unicode_ID,Full Name,Team,0,end_elo,start_elo,elo_change
576,5051515156,James Blackwell,Hurricanes,"(100.88391371265058, 106.42990302317858, -5.54...",100.883914,106.429903,-5.545989
826,5055485550,Salesi Rayasi,Hurricanes,"(110.70188334521939, 115.67822631062886, -4.97...",110.701883,115.678226,-4.976343
1070,5149525654,Quinn Tupaea,Chiefs,"(113.26707089934938, 118.08496318931884, -4.81...",113.267071,118.084963,-4.817892
267,5048485452,Ardie Savea,Hurricanes,"(118.33886733853684, 122.95562106235826, -4.61...",118.338867,122.955621,-4.616754
589,5051515748,Josh Ioane,Chiefs,"(98.22976447566921, 102.77928260508381, -4.549...",98.229764,102.779283,-4.549518
...,...,...,...,...,...,...,...
406,5049535052,Mattia Bellini,Benetton Treviso,"(96.79422627952788, 92.30762615721996, 4.48660...",96.794226,92.307626,4.486600
745,5053505254,Rhyno Smith,Benetton Treviso,"(99.09602275738854, 94.60942263508062, 4.48660...",99.096023,94.609423,4.486600
244,4957555455,Juan Ignacio Brex,Benetton Treviso,"(98.82188475913395, 94.33528463682603, 4.48660...",98.821885,94.335285,4.486600
408,5049535155,Federico Ruzza,Benetton Treviso,"(102.4315904715746, 97.94499034926667, 4.48660...",102.431590,97.944990,4.486600
